В процессе изучения задачи классификации текста нам стало интересно попробовать что-то ещё, к тому же классификацию отзывов о кофейнях Starbucks мы исследовали достаточно хорошо, попробовали разные методы: по-разному очищали данные, по-разному векторизовали текст (использовали BOW, TF-IDF, и прочее), наконец, использовали разные алгоритмы машинного обучения, даже к определению задачи по-разному подходили: рассматривали её и как задачу классификации на 5 классов, и как бинарную классификацию (в этом случае качество модели получилось намного лучше). Мы много слышали о том, что датасеты с Kaggle имеют мало отношения к реальности, поскольку датасеты уже подготовлены, максимум нужно удалить строки, где отсутствует информация. Поэтому мы решили придумать задачу, где датасет собрать реально, данные находятся в открытом доступе, но при этом со сбором надо повозиться. Было решено заняться задачей предсказания оценки за курс Машинное обучение 1 на основании оценок студентов за прошедшие математические курсы: матанализ, линал, теорвер, матстат, и другие. Особенность, как вы уже догадались, заключается в том, что оценки за курсы находятся в разных табличках, поэтому датасет придётся собрать самим, найдя оценки каждого студента в разных таблицах.

# Создание датасета

Начнём собирать данные. Для удобства давайте подключимся к гугл-диску, где создадим папку, где будем хранить все необходимые файлы:

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import os

directory = '/content/drive/My Drive/colab_data/mo-1'

if not os.path.exists(directory):
  os.makedirs(directory)

Оценки будем предсказывать за курс МО-1 2023/2024.

In [ ]:
import pandas as pd

Загрузим табличку с оценками 211 группы, чтобы посмотреть на данные. Сразу удалим все столбцы, кроме столбцов с ФИО и оценкой. Также заменим название столбца '❤️ ФИО ❤️' на 'ФИО', поскольку в других таблицах используется именно такое обозначение.

In [ ]:
mo_1_2023_211 = pd.read_csv('/content/drive/My Drive/colab_data/mo-1/mo-1_211.csv')

mo_1_2023_211 = mo_1_2023_211[['❤️ ФИО ❤️', 'Итог']]

print(mo_1_2023_211.head())

mo_1_2023_211.rename(columns={'❤️ ФИО ❤️': 'ФИО'}, inplace=True)

mo_1_2023_211 = mo_1_2023_211.drop(index=0)
mo_1_2023_211 = mo_1_2023_211[ : -3]

                          ❤️ ФИО ❤️  Итог
0                               NaN   NaN
1       Августёнок Алина Алексеевна   9.0
2     Аксененко Вероника Алексеевна   7.0
3  Барымов Илья Александрович (ПАД)   6.0
4      Гвоздева Дарья Александровна   5.0


В итоге мы заменили название столбца, который отвечает за ФИО для более удобной работы в дальнейшем, также удалили нулевую строку, потому что она не несёт в себе никакой информации, также удалили три последние строки, так как там была собрана всякая статистика по оценкам для преподавателей. Можем посмотреть, что получилось, остались две колонки с ФИО и оценками. Заметим, что в таблице могут присутствовать люди не с ПМИ, а для них проблематично будет собрать их оценки, поэтому в дальнейшем, в процессе сбора оценок, они отсеятся.

In [ ]:
print(mo_1_2023_211.head())

                                ФИО  Итог
1       Августёнок Алина Алексеевна   9.0
2     Аксененко Вероника Алексеевна   7.0
3  Барымов Илья Александрович (ПАД)   6.0
4      Гвоздева Дарья Александровна   5.0
5       Гриценко Дмитрий Витальевич   9.0


Теперь сделаем то же самое с оценками других групп. Напишем для этого цикл, который будет при необходимости менять название первого столбца на 'ФИО', а также удалять ненужные столбцы, удалять ненужные строки, и мерджить данные в единую табличку.

In [ ]:
groups = ['212', '213', '214_217', '215', '216', '218', '2110']

for group in groups:
  current_name_of_file = 'mo-1_' + group + '.csv'
  current_address = '/content/drive/My Drive/colab_data/mo-1/' + current_name_of_file
  current_file = pd.read_csv(current_address)

  current_name_of_fio = current_file.columns[0]
  if current_name_of_fio != 'ФИО':
    current_file.rename(columns={current_name_of_fio: 'ФИО'}, inplace=True)

  current_file = current_file.drop(index=0)
  current_file = current_file[ : -3]

  current_file = current_file[['ФИО', 'Итог']]
  mo_1_2023_211 = pd.concat([mo_1_2023_211, current_file], ignore_index=True)

print(mo_1_2023_211.head())

                                ФИО Итог
0       Августёнок Алина Алексеевна  9.0
1     Аксененко Вероника Алексеевна  7.0
2  Барымов Илья Александрович (ПАД)  6.0
3      Гвоздева Дарья Александровна  5.0
4       Гриценко Дмитрий Витальевич  9.0


Удалим пустые строки:

In [ ]:
mo_1 = mo_1_2023_211.dropna()
print(mo_1.head())

                                ФИО Итог
0       Августёнок Алина Алексеевна  9.0
1     Аксененко Вероника Алексеевна  7.0
2  Барымов Илья Александрович (ПАД)  6.0
3      Гвоздева Дарья Александровна  5.0
4       Гриценко Дмитрий Витальевич  9.0


Данные за курс МО-1 получены, теперь давайте собирать оценки за другие курсы, начнём с оценок первого года обучения: линал, матан, помним, что человек может быть как с основного потока, так и с пилотного, а также студент мог учиться сначала в основе, а потом в пилоте, или наоборот, поэтому будем это учитывать, а при наличии оценок и за пилотный курс, и за курс основы, будем брать из этого максимум, поскольку возможна ситуация, что студент начал проходить пилотный курс, но перевёлся в основу, но остался в табличке пилотого потока тоже.

## Линал
**Тут удобно, что оценки за курс находятся в таблицах одного формата, это облегчает работу.**

In [ ]:
linal_211 = pd.read_csv('/content/drive/My Drive/colab_data/mo-1/LAaG_211.csv')

linal_211 = linal_211[['Фамилия, имя', 'Овед']]

print(linal_211.head())

       Фамилия, имя  Овед
0     Акимов Сергей     7
1   Асатрян Георгий     9
2     Башарин Денис     9
3  Васильевых Павел     6
4        Гусев Иван     7


Делаем то же самое с остальными 12 файлами и сливаем их в один большой циклом:

In [ ]:
groups = ['212', '213', '214', '215', '216', '217', '218', '219', '2110', '2111', '2112']

for group in groups:
  current_name_of_file = 'LAaG_' + group + '.csv'
  current_address = '/content/drive/My Drive/colab_data/mo-1/' + current_name_of_file
  current_file = pd.read_csv(current_address)

  current_file = current_file[['Фамилия, имя', 'Овед']]

  linal_211 = pd.concat([linal_211, current_file], ignore_index=True)

print(linal_211.head())

       Фамилия, имя  Овед
0     Акимов Сергей   7.0
1   Асатрян Георгий   9.0
2     Башарин Денис   9.0
3  Васильевых Павел   6.0
4        Гусев Иван   7.0


In [ ]:
linal = linal_211.dropna()

linal.rename(columns={'Овед': 'Итог'}, inplace=True)

print(linal.head())

       Фамилия, имя  Итог
0     Акимов Сергей   7.0
1   Асатрян Георгий   9.0
2     Башарин Денис   9.0
3  Васильевых Павел   6.0
4        Гусев Иван   7.0


<ipython-input-11-86c9bdd92860>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  linal.rename(columns={'Овед': 'Итог'}, inplace=True)


## Матан

Начнём с таблицы пилотного потока:

In [ ]:
ma_1_pilot = pd.read_csv('/content/drive/My Drive/colab_data/mo-1/ma-1_pilot.csv')

ma_1_pilot = ma_1_pilot[['ФИО', 'Округление']]
ma_1_pilot = ma_1_pilot.dropna()

ma_1_pilot.rename(columns={'Округление': 'Итог'}, inplace=True)

print(ma_1_pilot.head())

                          ФИО  Итог
0     Акимов Сергей Вадимович   7.0
1  Асатрян Георгий Оганесович   9.0
2    Башарин Денис Витальевич   8.0
3            Гусев Иван Ильич   8.0
4      Дамиров Намиг Шаигович   8.0


Теперь основной поток:

In [ ]:
ma_1_based_213 = pd.read_csv('/content/drive/My Drive/colab_data/mo-1/МА-1 (2 сем) - 213.csv')

ma_1_based_213 = ma_1_based_213[['213 группа', 'Итог']]

ma_1_based_213 = ma_1_based_213.drop(index=0)
ma_1_based_213 = ma_1_based_213.drop(index=1)

ma_1_based_213.rename(columns={'213 группа': 'ФИО'}, inplace=True)

ma_1_based_213 = ma_1_based_213.dropna()

print(ma_1_based_213.head())

                              ФИО Итог
2         Бонич Дмитрий Сергеевич    8
3  Ведерников Константин Игоревич    7
4         Галкин Максим Сергеевич    7
5    Гапонов Александр Леонидович    8
6          Гаязов Булат Ринатович    6


Собираем общую таблицу:

In [ ]:
groups = ['215', '216', '217', '218', '219', '2110', '2111', '2112']

for group in groups:
  current_name_of_file = 'МА-1 (2 сем) - ' + group + '.csv'
  current_address = '/content/drive/My Drive/colab_data/mo-1/' + current_name_of_file
  current_file = pd.read_csv(current_address)

  current_file = current_file.drop(index=0)
  current_file = current_file.drop(index=1)

  current_name_of_fio = current_file.columns[1]
  if current_name_of_fio != 'ФИО':
    current_file.rename(columns={current_name_of_fio: 'ФИО'}, inplace=True)

  current_file = current_file[['ФИО', 'Итог']]

  ma_1_based_213 = pd.concat([ma_1_based_213, current_file], ignore_index=True)

In [ ]:
ma_1_based = ma_1_based_213.dropna()

print(ma_1_based.head())

                              ФИО Итог
0         Бонич Дмитрий Сергеевич    8
1  Ведерников Константин Игоревич    7
2         Галкин Максим Сергеевич    7
3    Гапонов Александр Леонидович    8
4          Гаязов Булат Ринатович    6


Общая таблица по матану:

In [ ]:
ma_1 = pd.concat([ma_1_based, ma_1_pilot], ignore_index=True)

print(ma_1.head())

                              ФИО Итог
0         Бонич Дмитрий Сергеевич    8
1  Ведерников Константин Игоревич    7
2         Галкин Максим Сергеевич    7
3    Гапонов Александр Леонидович    8
4          Гаязов Булат Ринатович    6


## Теорвер

Пилот. Тут будет неприятно, поскольку в колонке "Итог" стоят неокруглённые оценки, причём целая часть и дробная разделены запятой, а не точкой, поэтому меняем запятую на точку, а потом округляем.

In [ ]:
tv_211 = pd.read_csv('/content/drive/My Drive/colab_data/mo-1/tv_211.csv')
tv_212 = pd.read_csv('/content/drive/My Drive/colab_data/mo-1/tv_212.csv')
tv_214 = pd.read_csv('/content/drive/My Drive/colab_data/mo-1/tv_214.csv')

tv_211 = tv_211[['ФИО', 'Итог']]
tv_211 = tv_211.drop(index=0)
tv_211['Итог'] = tv_211['Итог'].str.replace(',', '.').astype(float)
tv_211['Итог'] = tv_211['Итог'].round()

tv_212 = tv_212[['ФИО', 'Итог']]
tv_212 = tv_212.drop(index=0)
tv_212['Итог'] = tv_212['Итог'].str.replace(',', '.').astype(float)
tv_212['Итог'] = tv_212['Итог'].round()

tv_214 = tv_214[['ФИО', 'Итог']]
tv_214 = tv_214.drop(index=0)
tv_214['Итог'] = tv_214['Итог'].str.replace(',', '.').astype(float)
tv_214['Итог'] = tv_214['Итог'].round()

tv_pilot = pd.concat([tv_211, tv_212, tv_214], ignore_index=True)

In [ ]:
tv_pilot = tv_pilot.dropna()
print(tv_pilot.head())

                             ФИО  Итог
0        Акимов Сергей Вадимович   7.0
1     Асатрян Георгий Оганесович  10.0
2       Башарин Денис Витальевич   9.0
3      Васильевых Павел Павлович   6.0
4  Горохов Антон Андреевич (215)   8.0


Основа:

In [ ]:
tv_based_213 = pd.read_csv('/content/drive/My Drive/colab_data/mo-1/ТВ-2022 - 213.csv')

tv_based_213 = tv_based_213[['213 группа', 'Итог']]
tv_based_213 = tv_based_213.drop(index=0)
tv_based_213.rename(columns={'213 группа': 'ФИО'}, inplace=True)
tv_based_213 = tv_based_213.dropna()

print(tv_based_213.head())

                              ФИО Итог
1            Баранов Даниил Ильич    8
2          Бекетов Егор Данилович    8
3         Бонич Дмитрий Сергеевич    0
4  Ведерников Константин Игоревич    6
5         Галкин Максим Сергеевич    0


In [ ]:
groups = ['215', '216', '217', '218', '219', '2110']

for group in groups:
  current_name_of_file = 'ТВ-2022 - ' + group + '.csv'
  current_address = '/content/drive/My Drive/colab_data/mo-1/' + current_name_of_file
  current_file = pd.read_csv(current_address)

  current_name_of_fio = current_file.columns[1]
  if current_name_of_fio != 'ФИО':
    current_file.rename(columns={current_name_of_fio: 'ФИО'}, inplace=True)

  current_file = current_file[['ФИО', 'Итог']]

  tv_based_213 = pd.concat([tv_based_213, current_file], ignore_index=True)

print(tv_based_213.head())

                              ФИО Итог
0            Баранов Даниил Ильич    8
1          Бекетов Егор Данилович    8
2         Бонич Дмитрий Сергеевич    0
3  Ведерников Константин Игоревич    6
4         Галкин Максим Сергеевич    0


In [ ]:
tv = pd.concat([tv_pilot, tv_based_213], ignore_index=True)
tv = tv.dropna()

print(tv.head())

                             ФИО  Итог
0        Акимов Сергей Вадимович   7.0
1     Асатрян Георгий Оганесович  10.0
2       Башарин Денис Витальевич   9.0
3      Васильевых Павел Павлович   6.0
4  Горохов Антон Андреевич (215)   8.0


## Матстат

Тут повезло, что табличка одна и для основы, и для пилота, ура:

In [ ]:
ms_211 = pd.read_csv('/content/drive/My Drive/colab_data/mo-1/Статистика 2022_2023 - Итог 211.csv')

ms_211 = ms_211[['ФИО', 'Итог']]
ms_211 = ms_211.drop(index=0)
ms_211['Итог'] = ms_211['Итог'].str.replace(',', '.').astype(float)
ms_211['Итог'] = ms_211['Итог'].round()

print(ms_211.head())

                          ФИО  Итог
1     Акимов Сергей Вадимович   4.0
2  Асатрян Георгий Оганесович   7.0
3    Башарин Денис Витальевич   9.0
4   Васильевых Павел Павлович   5.0
5            Гусев Иван Ильич   5.0


In [ ]:
groups = ['212', '213', '214', '215', '216', '217', '218', '219', '2110']

for group in groups:
  current_name_of_file = 'Статистика 2022_2023 - Итог ' + group + '.csv'
  current_address = '/content/drive/My Drive/colab_data/mo-1/' + current_name_of_file
  current_file = pd.read_csv(current_address)

  current_file = current_file[['ФИО', 'Итог']]

  current_file = current_file.drop(index=0)
  current_file['Итог'] = current_file['Итог'].str.replace(',', '.').astype(float)
  current_file['Итог'] = current_file['Итог'].round()

  ms_211 = pd.concat([ms_211, current_file], ignore_index=True)

print(ms_211.head())

                          ФИО  Итог
0     Акимов Сергей Вадимович   4.0
1  Асатрян Георгий Оганесович   7.0
2    Башарин Денис Витальевич   9.0
3   Васильевых Павел Павлович   5.0
4            Гусев Иван Ильич   5.0


In [ ]:
ms = ms_211.dropna()
print(ms.head())

                          ФИО  Итог
0     Акимов Сергей Вадимович   4.0
1  Асатрян Георгий Оганесович   7.0
2    Башарин Денис Витальевич   9.0
3   Васильевых Павел Павлович   5.0
4            Гусев Иван Ильич   5.0


Ура, у нас есть таблички с оценками по линалу, матану, теорверу, матстату всех групп в одном формате: ФИО, округлённый итог.

Снова переходим в табличку МО-1:

После удаления столбцов без информации могла сбиться нумерация, поэтому восстановим нормальную нумерацию, чтобы итерироваться по таблице и не ловить ошибки. Тут, кстати, не сбилась, но спокойно могла.

In [ ]:
mo_1.reset_index(drop=True, inplace=True)
print(mo_1)

                                     ФИО  Итог
0            Августёнок Алина Алексеевна   9.0
1          Аксененко Вероника Алексеевна   7.0
2       Барымов Илья Александрович (ПАД)   6.0
3           Гвоздева Дарья Александровна   5.0
4            Гриценко Дмитрий Витальевич   9.0
..                                   ...   ...
294  Григорьева Василиса Алексеевна (ПИ)   9.0
295    Смирнов Владислав Михайлович (ПИ)  10.0
296     Степашкина Виталия Павловна (ПИ)   0.0
297  Мостыка Николай Русланович (2 курс)   0.0
298                            Евзман Ян   4.0

[299 rows x 2 columns]


Заполняем табличку оценками по линалу:

In [ ]:
mo_1['linal'] = None
linal.reset_index(drop=True, inplace=True)

for index_1 in range(len(mo_1)):
  fio_1 = mo_1.at[index_1, 'ФИО']

  fio_1 = fio_1.split()

  for index_2 in range(len(linal)):
    fio_2 = linal.at[index_2, 'Фамилия, имя']

    fio_2 = fio_2.split()

    if fio_1[0] + fio_1[1] == fio_2[0] + fio_2[1]:
      mo_1.at[index_1, 'linal'] = linal.at[index_2, 'Итог']

<ipython-input-29-67ee2bac8a13>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  mo_1['linal'] = None


In [ ]:
yes, no = 0, 0

for i in mo_1['linal']:
  if i == None:
    no += 1
  else:
    yes += 1

print(yes, no)

232 67


Есть 232 оценки, нет 67 оценок. Связано это с тем, что в таблице находились люди не только с ПМИ, но и с других программ: ПИ, БИ, ИБ, ИВТ и так далее.

In [ ]:
mo_1['matan'] = None

<ipython-input-31-c2d5abbc50fb>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  mo_1['matan'] = None


In [ ]:
ma_1.reset_index(drop=True, inplace=True)

for index_1 in range(len(mo_1)):
  fio_1 = mo_1.at[index_1, 'ФИО']

  fio_1 = fio_1.split()

  for index_2 in range(len(ma_1)):
    fio_2 = ma_1.at[index_2, 'ФИО']

    if isinstance(fio_2, str):
      fio_2 = fio_2.split()

      if fio_1[0] + fio_1[1] == fio_2[0] + fio_2[1]:
        mo_1.at[index_1, 'matan'] = ma_1.at[index_2, 'Итог']

In [ ]:
print(mo_1)

                                     ФИО  Итог linal matan
0            Августёнок Алина Алексеевна   9.0   5.0     6
1          Аксененко Вероника Алексеевна   7.0   7.0     8
2       Барымов Илья Александрович (ПАД)   6.0   4.0     4
3           Гвоздева Дарья Александровна   5.0  None  None
4            Гриценко Дмитрий Витальевич   9.0   6.0     8
..                                   ...   ...   ...   ...
294  Григорьева Василиса Алексеевна (ПИ)   9.0  None  None
295    Смирнов Владислав Михайлович (ПИ)  10.0  None  None
296     Степашкина Виталия Павловна (ПИ)   0.0  None  None
297  Мостыка Николай Русланович (2 курс)   0.0   0.0     1
298                            Евзман Ян   4.0  None  None

[299 rows x 4 columns]


In [ ]:
yes, no = 0, 0

for i in mo_1['matan']:
  if i == None:
    no += 1
  else:
    yes += 1

print(yes, no)

233 66


Тут на одну оценку больше есть, кто-то остался без оценки по линалу :(((

In [ ]:
mo_1['teorver'] = None

<ipython-input-35-3f915b214004>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  mo_1['teorver'] = None


In [ ]:
tv.reset_index(drop=True, inplace=True)

for index_1 in range(len(mo_1)):
  fio_1 = mo_1.at[index_1, 'ФИО']

  fio_1 = fio_1.split()

  for index_2 in range(len(tv)):
    fio_2 = tv.at[index_2, 'ФИО']

    fio_2 = fio_2.split()

    if fio_1[0] + fio_1[1] == fio_2[0] + fio_2[1]:
      mo_1.at[index_1, 'teorver'] = tv.at[index_2, 'Итог']

In [ ]:
mo_1['matstat'] = None

<ipython-input-37-5a1b48b7f293>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  mo_1['matstat'] = None


In [ ]:
ms.reset_index(drop=True, inplace=True)

for index_1 in range(len(mo_1)):
  fio_1 = mo_1.at[index_1, 'ФИО']

  fio_1 = fio_1.split()

  for index_2 in range(len(ms)):
    fio_2 = ms.at[index_2, 'ФИО']

    fio_2 = fio_2.split()

    if len(fio_2) > 1:
      if fio_1[0] + fio_1[1] == fio_2[0] + fio_2[1]:
        mo_1.at[index_1, 'matstat'] = ms.at[index_2, 'Итог']

In [ ]:
print(mo_1)

                                     ФИО  Итог linal matan teorver matstat
0            Августёнок Алина Алексеевна   9.0   5.0     6       6     9.0
1          Аксененко Вероника Алексеевна   7.0   7.0     8       6     7.0
2       Барымов Илья Александрович (ПАД)   6.0   4.0     4    None    None
3           Гвоздева Дарья Александровна   5.0  None  None       5     5.0
4            Гриценко Дмитрий Витальевич   9.0   6.0     8       5     7.0
..                                   ...   ...   ...   ...     ...     ...
294  Григорьева Василиса Алексеевна (ПИ)   9.0  None  None    None    None
295    Смирнов Владислав Михайлович (ПИ)  10.0  None  None    None    None
296     Степашкина Виталия Павловна (ПИ)   0.0  None  None    None    None
297  Мостыка Николай Русланович (2 курс)   0.0   0.0     1       0    None
298                            Евзман Ян   4.0  None  None    None     4.0

[299 rows x 6 columns]


Теперь удалим строки, где нет оценок, и получим чистый датасет, состоящий из оценки за МО-1 и 4 оценок за математические курсы.

In [ ]:
data = mo_1.dropna()
data = data[['Итог', 'linal', 'matan', 'teorver', 'matstat']]

data = data[data['Итог'] != '#VALUE!']

In [ ]:
data.reset_index(drop=True, inplace=True)

data['Итог'] = data['Итог'].astype(float)
data['linal'] = data['linal'].astype(float)
data['matan'] = data['matan'].astype(float)
data['teorver'] = data['teorver'].astype(float)
data['matstat'] = data['matstat'].astype(float)

print(data)

     Итог  linal  matan  teorver  matstat
0     9.0    5.0    6.0      6.0      9.0
1     7.0    7.0    8.0      6.0      7.0
2     9.0    6.0    8.0      5.0      7.0
3    10.0    8.0    8.0      9.0      8.0
4     9.0    4.0    5.0      7.0      7.0
..    ...    ...    ...      ...      ...
220   8.0    7.0    7.0      6.0      5.0
221   8.0    5.0    5.0      6.0      5.0
222   6.0    7.0    5.0      6.0      4.0
223   7.0    4.0    5.0      4.0      5.0
224   4.0    6.0    4.0      4.0      4.0

[225 rows x 5 columns]


Датасет полностью готов. Давайте разделим его на признаки и целевую переменную:

In [ ]:
import numpy as np

In [ ]:
X = data.drop(columns=['Итог'])
X = X.values

y = data['Итог']
y = y.values

print(X.shape)
print(y.shape)

(225, 4)
(225,)


Делим выборку на тренировчную и тестовую, используем пропорцию 80 на 20:

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=10)

In [ ]:
print(X_train.shape, X_test.shape)
print(y_train.shape, y_test.shape)

(180, 4) (45, 4)
(180,) (45,)


# Обучение

**Начинаем обучение, наконец-то:**

Поскольку мы решаем задачу регрессии, предсказываем оценку от 0 до 10 за курс, не очень аргументированно будет оценивать качество модели с помощью accuracy: ну, потому что accuracy оценивает долю в точности правильных ответов, нас же скорее интересует не это, а среднее отклонение предсказанной оценки от фактической, поэтому логично использовать известную нам метрику MAE:

In [ ]:
from sklearn.metrics import mean_absolute_error

№1. Метод опорных векторов

In [ ]:
from sklearn import svm

SVM_model = svm.SVC()
SVM_model.fit(X_train, y_train)
SVM_prediction = SVM_model.predict(X_test)

print(mean_absolute_error(y_test, SVM_prediction))

1.288888888888889


Среднее отклонение получилось 1.28, что, на наш взгляд, является неплохим результатом, поскольку разброс оценок $< 1,5$ является довольно естественным, но об этом подробнее мы поговорим чуть позже, когда будем делать общие выводы.

№2. KNN

In [ ]:
from sklearn.neighbors import KNeighborsClassifier

KNN_model = KNeighborsClassifier(n_neighbors=5)
KNN_model.fit(X_train, y_train)
KNN_prediction = KNN_model.predict(X_test)
print(mean_absolute_error(y_test, KNN_prediction))

1.9555555555555555


KNN показал себя не очень хорошо: отклонение чуть меньше 2 баллов, что заметно хуже.

№3. Линейная регрессия

In [ ]:
from sklearn.linear_model import LinearRegression

LR_model = LinearRegression()
LR_model.fit(X_train, y_train)
LR_prediction = LR_model.predict(X_test)
print(mean_absolute_error(y_test, LR_prediction))

1.623966843457774


Лучше, чем KNN, но хуже, чем SVM.

Сейчас мы попробуем три различных линейных регрессии с регуляризациями, а потом объясним, что всё это значит:

№4. Lasso

In [ ]:
from sklearn.linear_model import Lasso

Lasso_model = Lasso(alpha=0.5)
Lasso_model.fit(X_train, y_train)
Lasso_prediction = Lasso_model.predict(X_test)
print(mean_absolute_error(y_test, Lasso_prediction))

1.6487063072471275


Такое себе.

№5. Ridge

In [ ]:
from sklearn.linear_model import Ridge

Ridge_model = Ridge(alpha=0.5)
Ridge_model.fit(X_train, y_train)
Ridge_prediction = Ridge_model.predict(X_test)
print(mean_absolute_error(y_test, Ridge_prediction))

1.6238806646816284


Чуть-чуть лучше.

№6. ElasticNet

In [ ]:
from sklearn.linear_model import ElasticNet

ElasticNet_model = ElasticNet(alpha=0.2, l1_ratio=0.8)
ElasticNet_model.fit(X_train, y_train)
ElasticNet_prediction = ElasticNet_model.predict(X_test)
print(mean_absolute_error(y_test, ElasticNet_prediction))

1.6044080489056995


Тут получилось 1.6, лучший результат, если не считать SVM. Lasso и Ridge - виды линейных регрессий с L-1 и L-2 регуляризациями, то есть, грубо говоря, со штрафами за неправильные ответы в процессе обучения, а ElasticNet - смесь Lasso и Ridge, коэффициент l1_ratio отвечает за соотношение L-1 и L-2 регуляризаций, и эта модель показала себя лучше других линейных регрессий.

№7. SVR

In [ ]:
from sklearn.svm import SVR

SVR_model = SVR(kernel='linear', C=2, gamma='auto', epsilon=1.0)
SVR_model.fit(X_train, y_train)
SVR_prediction = SVR_model.predict(X_test)
print(mean_absolute_error(y_test, SVR_prediction))

1.5688177648406656


Неплохо, кстати, работает метод SVR - регрессия, основанная на методе опорных векторов.

№8. Градиентный бустинг

In [ ]:
from sklearn.ensemble import GradientBoostingRegressor

GBR = GradientBoostingRegressor(n_estimators=150, learning_rate=0.47, max_depth=7, random_state=150)
GBR.fit(X_train, y_train)
GBR_prediction = GBR.predict(X_test)
print(mean_absolute_error(y_test, GBR_prediction))

1.6743322664798153


Очень скромно.

В общем, лучший результат у метода опорных векторов, MAE = 1.29, это самая небольшая из всех ошибок.

По опыту знаем, что задача бинарной классификации решается лучше, чем задача многоклассовой классификации или просто регрессии, поэтому, давайте проведём ещё один эксперимент, и будем предсказывать не оценку, а то, получил ли студент зачёт или словил незач.

In [ ]:
X_binary = np.where(X >= 4, 1, 0)
y_binary = np.where(y >= 4, 1, 0)

In [ ]:
X_2_train, X_2_test, y_2_train, y_2_test = train_test_split(X_binary, y_binary, test_size=0.20, random_state=20)

In [ ]:
from sklearn.metrics import accuracy_score

SVM_2_model = svm.SVC()
SVM_2_model.fit(X_2_train, y_2_train)
SVM_2_prediction = SVM_2_model.predict(X_2_test)

print(accuracy_score(y_2_test, SVM_2_prediction))

0.9777777777777777


Accuracy 0.98, но мы и по прошлому опыту знаем, что в задачах бинарной классификации качество очень сильно лучше, определять, получит студент зачёт или отправится на пересдачу, можно с достаточно высокой точностью :)

**Выводы.**

Какие выводы можно сделать?

Ну, во-первых, что быть настоящим дата-саентистом очень тяжело, потому что когда нет готового датасета, становится сильно сложнее, приходится много возиться, чтобы самим его себе смастерить. А ведь есть ещё разные секретные данные, на которых можно что-то обучить, тогда же приходится подписывать кучу соглашений и следить за тем, чтобы ничего никуда не утекло, в общем, тяжело.

Во-вторых, что бинарная классификация работает лучше всего, это понятно, accuracy 98% сам за себя всё говорит.

В-третьих, что метод опорных векторов работает лучше всего в задаче регрессии (предсказание оценок от 0 до 10), со средней ошибкой 1.28, что является очень неплохим результатом, поскольку предсказание оценки по оценкам за другие курсы не очень простая задача, потому что формирование оценки за курс зависит от многих факторов. Иногда и от везения, а бывают ситуации, когда студент плохо ботал 1 курс, но начал хорошо ботать на 2, а бывает так, что вообще только с 3 курса начал закрываться на отлы, также нельзя забывать о вопросах мотивации и прочем. Но даже несмотря на все эти факторы, модель построить удалось, и MAE 1.28 говорит об этом. Всё-таки, для прогнозирования оценки, которая может быть одним из 11 чисел, средняя ошибка 1.28 является довольно хорошим результатом, поэтому можно говорить о том, что оценка за курс студента по его предыдущим заслугам прикидывается с неплохой точностью.

#*блин, надеюсь это хоть кто-то прочитает...*